# Import Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# Import the datasets

In [25]:
peopleFilepath = './Data/pickledPeople.pkl'
voteFilepath = './Data/pickledVotes_unpacked.pkl'
routeFilepath = './Data/pickledRoutes_unpacked.pkl'

people = pd.read_pickle(peopleFilepath)
votes = pd.read_pickle(voteFilepath)
routes = pd.read_pickle(routeFilepath)

## Function to find common people for an area

In [10]:
# Function to find all common people between N most-popular routes
def findCommonPeople(areaUrl, breadcrumbTier, nRoutes=10, routes=routes, people=people):
    areaRoutes = routes.loc[routes[breadcrumbTier]==areaUrl]
    areaRoutes = areaRoutes.sort_values('numQualityVotes', ascending=False)
    areaRoutes = areaRoutes[:nRoutes]
    
    subPeople = set([])
    
    # Filter the people dataset to only include those with at least nRoutes starRatings
    people = people.loc[people['numStarRatings']>=nRoutes*5]
            
    for i in range(len(areaRoutes)):
        tempSet=[]
        print("Parsing peopleDF for route", i+1, "of", nRoutes)
        for j in range(len(people)):
            if areaRoutes.index[i] in people.iloc[j]['starRatings']:
                tempSet.append(people.index[j])
                print(j/len(people), end='\r')        
        if i==0:
            subPeople=set(tempSet)
        else:
            subPeople = subPeople.intersection(set(tempSet))
            
    print(len(subPeople),"have climbed the", nRoutes, "most-rated climbs at", areaUrl) 

    return subPeople

## Isolate the People to be used for classification

In [54]:
eldoUrl = 'https://www.mountainproject.com/area/105744246/eldorado-canyon-sp'
subpeople = findCommonPeople(eldoUrl, 'bc3')

Parsing peopleDF for route 1 of 10


Parsing peopleDF for route 2 of 10
Parsing peopleDF for route 3 of 10
Parsing peopleDF for route 4 of 10
Parsing peopleDF for route 5 of 10
Parsing peopleDF for route 6 of 10
Parsing peopleDF for route 7 of 10
Parsing peopleDF for route 8 of 10
Parsing peopleDF for route 9 of 10
Parsing peopleDF for route 10 of 10
95 have climbed the 10 most-rated climbs at https://www.mountainproject.com/area/105744246/eldorado-canyon-sp


## Isolate the Routes to be used for classification

In [55]:
eldo = routes.loc[routes['bc3']=='https://www.mountainproject.com/area/105744246/eldorado-canyon-sp']
eldo = eldo.sort_values(by='numQualityVotes', ascending=False)
eldo = eldo[:10]

## Separate People into Training and Testing datasets

In [138]:
# Make the training dataset
train = people.loc[subpeople[0]]
for i in range(1, int(len(subpeople)/2)):
    train = pd.concat([train, people.loc[subpeople[i]]], axis=1)
train = train.transpose()

# Make the test dataset
test = people.loc[subpeople[int(len(subpeople)/2)]]
for i in range(int(len(subpeople)/2)+1, len(subpeople)):
    test = pd.concat([test, people.loc[subpeople[i]]], axis=1)
test = test.transpose()

# All together, just in case it is needed
total = pd.concat([test, train])

TypeError: string indices must be integers

In [122]:
train

,starRatings,diffRatings,ticks,toDoList,numStarRatings,url
https://www.mountainproject.com/user/108244276/vrp,{'https://www.mountainproject.com/route/105717...,{},{},[https://www.mountainproject.com/route/1057173...,160,https://www.mountainproject.com/user/108244276...
https://www.mountainproject.com/user/107186883/dwight-jugornot,{'https://www.mountainproject.com/route/105714...,{},{'https://www.mountainproject.com/route/105714...,[https://www.mountainproject.com/route/1057175...,1002,https://www.mountainproject.com/user/107186883...
https://www.mountainproject.com/user/105841324/dane-casterson,{'https://www.mountainproject.com/route/105717...,{},{'https://www.mountainproject.com/route/105717...,[https://www.mountainproject.com/route/1057175...,495,https://www.mountainproject.com/user/105841324...
https://www.mountainproject.com/user/105890911/greg-davis,{'https://www.mountainproject.com/route/105714...,{},{'https://www.mountainproject.com/route/105714...,[https://www.mountainproject.com/route/1057173...,1912,https://www.mountainproject.com/user/105890911...
https://www.mountainproject.com/user/110545103/james-klaers,{'https://www.mountainproject.com/route/105717...,{},{'https://www.mountainproject.com/route/105715...,[https://www.mountainproject.com/route/1057173...,182,https://www.mountainproject.com/user/110545103...
https://www.mountainproject.com/user/111656276/dakota-millard,{'https://www.mountainproject.com/route/105732...,{},{'https://www.mountainproject.com/route/105732...,[https://www.mountainproject.com/route/1057471...,277,https://www.mountainproject.com/user/111656276...
https://www.mountainproject.com/user/11047/ross-keller,{'https://www.mountainproject.com/route/105717...,{},{'https://www.mountainproject.com/route/105749...,[],561,https://www.mountainproject.com/user/11047/ros...
https://www.mountainproject.com/user/111431823/mitch-friedeman,{'https://www.mountainproject.com/route/105717...,{},{'https://www.mountainproject.com/route/105717...,[https://www.mountainproject.com/route/1057176...,464,https://www.mountainproject.com/user/111431823...
https://www.mountainproject.com/user/107639626/jeremy-r,{'https://www.mountainproject.com/route/105714...,{},{'https://www.mountainproject.com/route/105714...,[https://www.mountainproject.com/route/1057177...,304,https://www.mountainproject.com/user/107639626...
https://www.mountainproject.com/user/107226326/ryan-kempf,{'https://www.mountainproject.com/route/105714...,{},{'https://www.mountainproject.com/route/105721...,[https://www.mountainproject.com/route/1059184...,285,https://www.mountainproject.com/user/107226326...


## Create an array of starValues

In [156]:
npTrain = np.zeros(shape=(10, len(train)), dtype=int)
for i in range(len(eldo)):
    for j in range(len(train)):
        npTrain[i][j] = train.iloc[j]['starRatings'][eldo.index[i]]

In [175]:
for i in range(len(npTrain)):
    print(eldo.index[i][:48], npTrain[i][0:30])

https://www.mountainproject.com/route/105748490/ [3 4 3 4 3 3 4 4 4 3 4 4 3 4 4 3 4 3 3 4 3 3 4 4 2 3 3 4 4 4]
https://www.mountainproject.com/route/105750106/ [4 3 4 4 4 4 3 4 4 4 4 4 3 4 4 2 4 3 4 4 4 3 4 4 3 4 3 4 3 4]
https://www.mountainproject.com/route/105748657/ [4 3 4 4 4 4 4 4 4 3 4 3 4 3 4 4 4 4 4 3 4 3 3 4 4 4 3 3 4 4]
https://www.mountainproject.com/route/105748361/ [2 2 3 3 4 4 3 4 3 3 3 3 3 4 4 3 3 3 3 3 2 3 3 3 3 3 3 3 4 3]
https://www.mountainproject.com/route/105748391/ [2 3 2 3 3 3 3 4 2 3 3 2 3 3 2 2 3 3 2 3 3 2 3 4 2 3 3 4 3 3]
https://www.mountainproject.com/route/105749890/ [4 3 3 4 4 3 3 3 4 3 4 3 3 2 4 2 3 3 3 4 3 2 4 3 3 3 3 4 3 4]
https://www.mountainproject.com/route/105748774/ [4 3 4 4 4 3 3 4 4 4 4 3 4 4 4 3 4 4 3 4 3 4 3 3 3 4 4 4 4 2]
https://www.mountainproject.com/route/105748639/ [4 3 4 4 4 3 3 4 4 4 3 3 3 3 4 2 4 3 3 4 3 3 4 3 3 4 4 4 4 4]
https://www.mountainproject.com/route/105748924/ [3 3 3 4 4 4 4 4 4 3 3 4 3 4 4 2 3 4 3 3 4 2 3 3 4 3 3 4 4 3]
h

### Visualization to see general star distribution

In [140]:
# Visualization to see general star distribution
d = {0:0, 1:0, 2:0, 3:0, 4:0}
r = list(eldo.index)
for i in range(len(total)):
    for j in range(len(eldo)):
        print(total.iloc[i]['starRatings'][r[j]], end=' ')
        d[total.iloc[i]['starRatings'][r[j]]] = d[total.iloc[i]['starRatings'][r[j]]]+1
    print()
    
for i in d:
    print(i, ":", d[i])

4 4 3 4 2 4 4 3 4 4 
4 4 4 3 4 3 4 4 3 4 
4 3 4 3 3 4 3 3 3 3 
3 4 4 3 3 3 4 4 2 4 
4 4 4 4 3 3 3 4 4 4 
3 3 4 2 2 3 2 2 2 3 
4 4 4 3 3 4 4 4 4 2 
3 3 4 4 2 3 3 3 3 4 
2 3 4 4 3 4 4 3 3 4 
3 3 2 3 2 3 3 2 2 3 
4 3 3 4 3 4 4 4 3 3 
4 3 4 4 3 3 4 4 4 4 
4 3 4 4 2 3 4 3 3 3 
3 4 3 3 3 3 4 4 3 4 
4 4 4 4 4 4 4 4 4 4 
3 3 4 3 3 3 4 4 3 3 
2 4 4 3 3 3 4 3 4 2 
4 4 4 4 3 3 4 4 3 3 
4 4 4 3 3 4 4 4 3 3 
3 4 4 2 3 4 4 4 4 4 
2 4 4 3 3 2 4 4 4 3 
4 4 4 4 3 3 4 4 3 4 
4 3 4 4 3 3 4 3 3 2 
3 3 4 2 3 3 4 4 3 4 
4 3 4 3 3 3 4 4 3 3 
3 3 4 2 3 3 4 3 3 3 
3 4 4 3 3 4 4 4 4 4 
4 4 4 3 3 4 4 4 4 4 
4 4 2 3 3 4 4 3 3 4 
4 4 4 4 3 3 4 4 4 4 
4 3 4 4 4 4 4 4 4 4 
4 4 4 3 3 3 3 3 3 3 
4 4 4 3 3 3 4 3 4 3 
3 3 4 3 3 3 3 3 3 3 
3 4 4 3 3 4 4 4 4 4 
4 4 4 2 2 3 4 3 2 3 
4 4 4 3 2 4 4 4 4 4 
3 4 3 2 3 3 4 4 2 3 
4 4 4 4 4 3 4 4 4 4 
4 4 4 3 4 4 4 3 4 4 
4 3 4 2 3 3 4 2 2 3 
4 4 4 3 3 4 4 4 4 4 
4 3 2 4 3 3 3 4 4 3 
2 3 3 2 3 3 4 3 4 4 
4 4 4 3 4 3 4 4 2 4 
4 3 4 3 2 3 4 3 4 3 
4 4 4 3 3 4 3 4 3 4 
4 3 4 4 3 4 4

In [133]:
test.iloc[0]['starRatings']['https://www.mountainproject.com/route/105748490/the-bastille-crack']

4

In [139]:
total

,starRatings,diffRatings,ticks,toDoList,numStarRatings,url
https://www.mountainproject.com/user/107820434/nathaniel-dray,{'https://www.mountainproject.com/route/105717...,{},{'https://www.mountainproject.com/route/105717...,[https://www.mountainproject.com/route/1057481...,494,https://www.mountainproject.com/user/107820434...
https://www.mountainproject.com/user/107937197/eddie-f,{'https://www.mountainproject.com/route/105715...,{},{'https://www.mountainproject.com/route/105715...,[https://www.mountainproject.com/route/1057148...,267,https://www.mountainproject.com/user/107937197...
https://www.mountainproject.com/user/13658/rob-bauer,{'https://www.mountainproject.com/route/105714...,{},{'https://www.mountainproject.com/route/105714...,[https://www.mountainproject.com/route/1057176...,1949,https://www.mountainproject.com/user/13658/rob...
https://www.mountainproject.com/user/106285189/david-a,{'https://www.mountainproject.com/route/105714...,{},{'https://www.mountainproject.com/route/105714...,[https://www.mountainproject.com/route/1057149...,100,https://www.mountainproject.com/user/106285189...
https://www.mountainproject.com/user/111146152/stephen-schilling,{'https://www.mountainproject.com/route/105717...,{},{'https://www.mountainproject.com/route/105717...,[https://www.mountainproject.com/route/1057173...,473,https://www.mountainproject.com/user/111146152...
https://www.mountainproject.com/user/106452465/goodbye,{'https://www.mountainproject.com/route/105714...,{},{'https://www.mountainproject.com/route/105714...,[https://www.mountainproject.com/route/1057149...,462,https://www.mountainproject.com/user/106452465...
https://www.mountainproject.com/user/12121/paul-s,{'https://www.mountainproject.com/route/105717...,{},{},[],416,https://www.mountainproject.com/user/12121/paul-s
https://www.mountainproject.com/user/105990845/orphaned,{'https://www.mountainproject.com/route/105714...,{},{},[],3693,https://www.mountainproject.com/user/105990845...
https://www.mountainproject.com/user/10232/tony-b,{'https://www.mountainproject.com/route/105714...,{},{'https://www.mountainproject.com/route/105714...,[https://www.mountainproject.com/route/1057177...,4035,https://www.mountainproject.com/user/10232/tony-b
https://www.mountainproject.com/user/105837552/patrick-kadel,{'https://www.mountainproject.com/route/105721...,{},{'https://www.mountainproject.com/route/105756...,[https://www.mountainproject.com/route/1057497...,243,https://www.mountainproject.com/user/105837552...


In [132]:
people.describe()

,numStarRatings
count,96920.000000
mean,19.434472
std,87.396775
min,0.000000
25%,0.000000
50%,0.000000
75%,6.000000
max,4814.000000
